In [10]:
import datetime

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# Importing custom libraries
import sys
sys.path.append('/utilities/')
from utilities.data_manipulation import pivot_dataframe, convert_to_supervised, rename_dataframe_supervised, plot_results

# Random seed for reproducibility
tf.random.set_seed(42)

REFRESH_DATA = False
DATA_QUANTITY = 200

### Loading Data

In [11]:
# Defining city id's
ids = ['1064', '1067']


# Creating 
combined_df = []
if REFRESH_DATA:
    air_measurements = []
    # Iterating through sensor locations and creating object to store data for each
    for location in ids:
        quality = AirQuality(location)
        quality.pull_data(DATA_QUANTITY )
        air_measurements.append(quality.raw_data)

    combined_df = pd.concat(air_measurements)
    combined_df.to_csv('./Data/air_polution.csv')
else:
    combined_df = pd.read_csv('./Data/air_polution.csv')
    combined_df = combined_df.drop(columns = ['Unnamed: 0'])

combined_df['location'] = combined_df['location'].astype(str)
combined_df = combined_df.drop(columns = ['Date'])
combined_df = combined_df.rename(columns = {'Data_Converted': 'Date'})
combined_df = combined_df[combined_df['location'].isin(ids)].reset_index(drop=True)
combined_df.head(10)

,Value,location,parameter,longitude,latitude,Date
0,2.8,1064,pm25,-122.602877,45.496641,2024-08-22 12:00:00+00:00
1,2.7,1064,pm25,-122.602877,45.496641,2024-08-22 11:00:00+00:00
2,2.6,1064,pm25,-122.602877,45.496641,2024-08-22 10:00:00+00:00
3,2.7,1064,pm25,-122.602877,45.496641,2024-08-22 09:00:00+00:00
4,2.6,1064,pm25,-122.602877,45.496641,2024-08-22 08:00:00+00:00
5,2.5,1064,pm25,-122.602877,45.496641,2024-08-22 07:00:00+00:00
6,2.4,1064,pm25,-122.602877,45.496641,2024-08-22 06:00:00+00:00
7,2.7,1064,pm25,-122.602877,45.496641,2024-08-22 05:00:00+00:00
8,2.7,1064,pm25,-122.602877,45.496641,2024-08-22 04:00:00+00:00
9,2.9,1064,pm25,-122.602877,45.496641,2024-08-22 03:00:00+00:00


In [12]:
identifier_col = 'location'
value_col = 'Value'
formatted_df = pivot_dataframe(combined_df, identifier_col, value_col)
formatted_df.head(10)

,1064,1067,Date
0,2.8,2.8,2024-08-22 12:00:00+00:00
1,2.7,2.8,2024-08-22 11:00:00+00:00
2,2.6,2.7,2024-08-22 10:00:00+00:00
3,2.7,2.6,2024-08-22 09:00:00+00:00
4,2.6,2.4,2024-08-22 08:00:00+00:00
5,2.5,2.5,2024-08-22 07:00:00+00:00
6,2.4,2.3,2024-08-22 06:00:00+00:00
7,2.7,2.6,2024-08-22 05:00:00+00:00
8,2.7,2.8,2024-08-22 04:00:00+00:00
9,2.9,2.9,2024-08-22 03:00:00+00:00


### First Trying with Just One Series

In [13]:
formatted_df = formatted_df[['1064', 'Date']]

# creating scalar
scaler = MinMaxScaler(feature_range=(0, 1))

# Normalizing target column
formatted_df ['1064'] = scaler.fit_transform(formatted_df[['1064']])
formatted_df.head(10)

,1064,Date
0,0.043478,2024-08-22 12:00:00+00:00
1,0.034783,2024-08-22 11:00:00+00:00
2,0.026087,2024-08-22 10:00:00+00:00
3,0.034783,2024-08-22 09:00:00+00:00
4,0.026087,2024-08-22 08:00:00+00:00
5,0.017391,2024-08-22 07:00:00+00:00
6,0.008696,2024-08-22 06:00:00+00:00
7,0.034783,2024-08-22 05:00:00+00:00
8,0.034783,2024-08-22 04:00:00+00:00
9,0.052174,2024-08-22 03:00:00+00:00


### Creating Training Set

In [14]:
total_observations = len(formatted_df)
train_ratio = 0.7

train_df = formatted_df[:int(total_observations * train_ratio)]
test_df = formatted_df[int(total_observations * train_ratio):]
print(len(train_df), len(test_df))


140 60


### Converting Data to Windowed Format

In [15]:
timesteps = 6
train_vals, test_vals = train_df['1064'].values, test_df['1064'].values

# Transforming to time series                                                                                                                                                       
train_data, test_data = convert_to_supervised(train_vals, n_in=timesteps), convert_to_supervised(test_vals, n_in=timesteps)

# Converting to dataframe
train_df, test_df = rename_dataframe_supervised(pd.DataFrame(train_data)), rename_dataframe_supervised(pd.DataFrame(test_data))

# Separating x and y
train_y, train_x = train_df['t'].to_numpy(), train_df.drop(columns = ['t']).to_numpy()
test_y, test_x = test_df['t'].to_numpy(), test_df.drop(columns = ['t']).to_numpy()


In [16]:
# reshape input to be [samples, time steps, features]
train_x = np.reshape(train_x, (train_x.shape[0], 1, train_x.shape[1]))
test_x = np.reshape(test_x, (test_x.shape[0], 1, test_x.shape[1]))


In [21]:
print('Train X shape:', train_x.shape)
print('Train y shape:', train_y.shape)

Train X shape: (134, 1, 6)
Train y shape: (134,)


### Now we can Make a Model

In [18]:
# defining and training model
model = Sequential()
model.add(LSTM(4, input_shape=(1, timesteps)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_x, train_y, epochs=100, batch_size=1, verbose=3)

# make predictions
trainPredict = model.predict(train_x)
testPredict = model.predict(test_x)


# inverting predictions back to initial scale
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([train_y])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([test_y])

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [19]:
# calculate root mean squared error
trainScore = np.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 0.65 RMSE
Test Score: 0.53 RMSE


### Plotting Results

In [ ]:
base_dataset = scaler.inverse_transform(formatted_df['1064'].values)
plot_results(trainPredict, testPredict, base_dataset, timesteps, scaler)